In [2]:
from googleapiclient.discovery import build
import sys; sys.path.append('.') # appending root directory
import os
from utils.download_yt import download
import argparse
import re

# normalizing 
def normalize_title(titles): # to handle youtube api sending weird title str for special character bug
    def normalize(title): 
        # Convert to lowercase
        title = title.lower()
        # Replace special characters and extra spaces
        title = re.sub(r'[^\w\s]', '', title)  # Remove special characters
        title = re.sub(r'\s+', ' ', title)     # Replace multiple spaces with a single space
        title = title.strip()                   # Trim leading/trailing spaces
        return title
    
    # normalizing only keys if dict
    if isinstance(titles, dict): 
        # empty dict to return 
        normalized_titles = {}
        
        for key, value in titles.items(): 
            normalized_titles.update({normalize(key): value})
        
        return normalized_titles
    
    # if other then dict
    if not isinstance(titles, list): 
        titles = list(titles)
        
    normalized_titles = []
    for title in titles: 
        normalized_titles.append(normalize(title))
    
    return normalized_titles
        

# Function to get videos from a playlist
def getYTPlaylistVideos(playlist_id):    
    # youtube api 
    from config.api import youtube_api as api_key
    
    # Build the YouTube API client
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    videos = {}
    request = youtube.playlistItems().list(
        part='snippet',
        playlistId=playlist_id,
        maxResults=50  # You can change this to fetch more results per request
    )
    
    while request:
        response = request.execute()

        for item in response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            video_title = item['snippet']['title']
            video_link = f'https://www.youtube.com/watch?v={video_id}'
            
            videos.update({video_title: video_link})

        # Check if there's a next page
        request = youtube.playlistItems().list_next(request, response)
    
    return videos

def getLocalMusics(sync_dict): 
    # initalize empty list to store musics name
    musics = []
    
    # Walk through the directory tree
    for _, _,  filenames in os.walk(sync_dict):
        # List files in the current directory
        for filename in filenames:
            musicTitle = os.path.splitext(filename)[0]
            musics.append(musicTitle)
    
    return musics

def sync(playlist_id, sync_dict): 
    if not os.path.exists(sync_dict): 
        os.makedirs(sync_dict)
        
    # Get the videos from the playlist
    videos = normalize_title(getYTPlaylistVideos(playlist_id))
    
    # Get local videos
    localMusics = normalize_title(getLocalMusics(sync_dict))
    
    # videos in remote only 
    remoteOnly = set(videos.keys()) - set(localMusics)

    for video in remoteOnly: 
        url = videos[video]
        download(url, sync_dict)

In [3]:
%timeit sync('PLAU44P-DKz39QYgl7VW4NGgYLLlIF-Unc', 'videos')

875 ms ± 51.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
